<a href="https://colab.research.google.com/github/katrinag2004/ds2002assignments/blob/main/dataproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import requests
import pandas as pd
import sqlite3

# Local CSV file containing college data
COLLEGE_DATA_FILE = "Most-Recent-Cohorts-Institution.csv"

# FBI crime data API base URL and API key (note: remove any extra suffix like .GOV)
BASE_URL = "https://data.cdc.gov/resource/xbxb-epbu.json"

# State abbreviations to full state names mapping
STATE_ABBR_TO_NAME = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

def fetch_cdc_data(params=None):
    """
    Fetches CDC data from the provided API endpoint.
    """
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json()
        # Normalize the JSON response into a pandas DataFrame
        return pd.json_normalize(data)
    except Exception as e:
        print(f"API Error: {str(e)}")
        return pd.DataFrame()

def fetch_college_data():
    """
    Loads college data from a CSV file.
    """
    try:
        return pd.read_csv(COLLEGE_DATA_FILE, low_memory=False)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return pd.DataFrame()

def clean_college_data(df):
    """
    Cleans the college data by selecting and renaming relevant columns.
    Also, converts state abbreviations to full state names.
    """
    try:
        columns_needed = ['INSTNM', 'CITY', 'STABBR', 'UGDS', 'C150_4', 'RET_FT4',
                          'UGDS_MEN', 'UGDS_WOMEN', 'TUITIONFEE_IN', 'DEBT_MDN',
                          'INEXPFTE', 'HIGHDEG', 'PRGMOFR']
        df = df[columns_needed].copy()
        df.columns = ['CollegeName', 'City', 'State', 'Enrollment', 'GraduationRate',
                      'RetentionRate', 'PercentUGMen', 'PercentUGWomen', 'InstateTuition',
                      'MedianTotalDebt', 'ExpendituresPerStudent', 'HighestDegreeAwarded',
                      'NumberProgramsOffered']

        # Replace state abbreviations with full state names
        df['State'] = df['State'].map(STATE_ABBR_TO_NAME)

        return df
    except Exception as e:
        print(f"Error cleaning college data: {e}")
        return pd.DataFrame()

def clean_cdc_data(df):
    """
    Cleans the CDC data by selecting and renaming columns.
    """
    if df.empty:
        return df
    try:
        # Select the relevant columns
        required_columns = ['state', 'crude_death_rate']  # Use the appropriate column name
        for col in required_columns:
            if col not in df.columns:
                print(f"Column {col} not found in CDC data.")
                return pd.DataFrame()
        df = df[required_columns].copy()
        df.columns = ['State', 'CrudeDeathRate']  # Renaming columns for consistency

        # Convert CrudeDeathRate to numeric, forcing errors to NaN
        df['CrudeDeathRate'] = pd.to_numeric(df['CrudeDeathRate'], errors='coerce')

        # Drop rows with NaN values in CrudeDeathRate
        df = df.dropna(subset=['CrudeDeathRate'])

        return df
    except Exception as e:
        print(f"Error cleaning CDC data: {e}")
        return pd.DataFrame()


def merge_data(college_df, cdc_df):
    """
    Merges the college and CDC data.

    The merge is performed on the 'State' column.
    """
    try:
        merged_df = pd.merge(college_df, cdc_df, on="State", how="inner")
        return merged_df
    except Exception as e:
        print(f"Error merging data: {e}")
        return pd.DataFrame()

def load_to_sqlite(df, db_name="college_cdc.db"):
    """
    Loads the merged data into a SQLite database.
    """
    try:
        with sqlite3.connect(db_name) as conn:
            df.to_sql("CollegeCDC", conn, if_exists="replace", index=False)
    except Exception as e:
        print(f"Error loading data to SQLite: {e}")

def generate_summary(df):
    """
    Generates and prints summary statistics of the merged dataset.
    """
    try:
        print("Summary Statistics:")
        print(df.describe(include='all'))
        print(f"Total records: {len(df)}")
        print(f"Columns: {list(df.columns)}")
    except Exception as e:
        print(f"Error generating summary: {e}")

def analyze_correlation(df):
    """
    Analyzes the correlation between the number of colleges and the death rate.
    """
    # Step 1: Count the number of colleges per state
    college_counts = df['State'].value_counts().reset_index()
    college_counts.columns = ['State', 'NumColleges']

    # Step 2: Get the average death rate per state
    death_rate_per_state = df.groupby('State')['CrudeDeathRate'].mean().reset_index()
    death_rate_per_state.columns = ['State', 'AverageDeathRate']

    # Step 3: Merge the two datasets on the 'State' column
    merged_data = pd.merge(college_counts, death_rate_per_state, on='State', how='inner')

    # Step 4: Calculate correlation
    correlation = merged_data['NumColleges'].corr(merged_data['AverageDeathRate'])

    # Print the result
    print(f"Correlation between the number of colleges and the death rate per state: {correlation}")

def run_pipeline():
    """
    Orchestrates the ETL process: reading, cleaning, merging, storing,
    and summarizing the college and CDC datasets.
    """
    print("Fetching college data...")
    college_df = fetch_college_data()
    if college_df.empty:
        print("College data not loaded. Exiting pipeline.")
        return
    college_df = clean_college_data(college_df)

    print("Fetching CDC data...")
    cdc_df = fetch_cdc_data(params={"$limit": 5000})  # Adjust parameters as needed
    if cdc_df.empty:
        print("CDC data not available. Exiting pipeline.")
        return
    cdc_df = clean_cdc_data(cdc_df)
    if cdc_df.empty:
        print("CDC data cleaning failed. Exiting pipeline.")
        return

    print("Merging data on State...")
    merged_df = merge_data(college_df, cdc_df)
    if merged_df.empty:
        print("Merging resulted in an empty dataset. Exiting pipeline.")
        return

    print("Loading merged data into SQLite database...")
    load_to_sqlite(merged_df)

    print("Generating summary...")
    generate_summary(merged_df)

    print("Analyzing correlation...")
    analyze_correlation(merged_df)

    print("✅ ETL Pipeline Completed Successfully!")

if __name__ == "__main__":
    run_pipeline()


Fetching college data...
Fetching CDC data...
Merging data on State...
Loading merged data into SQLite database...
Generating summary...
Summary Statistics:
              CollegeName      City       State     Enrollment  \
count              113472    113472      113472  100242.000000   
unique               6188      2338          50            NaN   
top     Cortiva Institute  New York  California            NaN   
freq                  108      1422       12384            NaN   
mean                  NaN       NaN         NaN    2452.156402   
std                   NaN       NaN         NaN    5904.402589   
min                   NaN       NaN         NaN       0.000000   
25%                   NaN       NaN         NaN     110.000000   
50%                   NaN       NaN         NaN     489.000000   
75%                   NaN       NaN         NaN    2091.000000   
max                   NaN       NaN         NaN  138138.000000   

        GraduationRate  RetentionRate   PercentUGM